In [3]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.regression.LinearRegression
import org.apache.log4j._

Logger.getLogger("org").setLevel(Level.ERROR)
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.regression.LinearRegression
import org.apache.log4j._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@76f4c67a


In [4]:
val data = spark.read.option("header", "true")
            .option("inferSchema", "true")
            .format("csv")
            .load("../datasets/Clean_Ecommerce.csv")
data.show()

+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|     SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313|   3.120178782748092|  581.8523440352177|
|mstephens@davidso...|MediumAquaMa

data: org.apache.spark.sql.DataFrame = [Email: string, Avatar: string ... 5 more fields]


In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.head(1)

res3: Array[org.apache.spark.sql.Row] = Array([mstephenson@fernandez.com,Violet,34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615,587.9510539684005])


In [7]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors

import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors


In [8]:
val df = data.select(data("Yearly Amount Spent").as("label"),
                     $"Avg Session Length",
                     $"Time on App",
                     $"Time on Website",
                     $"Length of Membership")
df.printSchema()

root
 |-- label: double (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)



df: org.apache.spark.sql.DataFrame = [label: double, Avg Session Length: double ... 3 more fields]


In [15]:
val assembler = new VectorAssembler()
                .setInputCols(Array("Avg Session Length", "Time on App", "Time on Website", "Length of Membership"))
                .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_7cbebd90b6bb, handleInvalid=error, numInputCols=4


In [18]:
val output = assembler.transform(df).select($"label", $"features")

output: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [19]:
output.show()

+------------------+--------------------+
|             label|            features|
+------------------+--------------------+
| 587.9510539684005|[34.4972677251122...|
| 392.2049334443264|[31.9262720263601...|
|487.54750486747207|[33.0009147556426...|
| 581.8523440352177|[34.3055566297555...|
| 599.4060920457634|[33.3306725236463...|
|  637.102447915074|[33.8710378793419...|
| 521.5721747578274|[32.0215955013870...|
| 549.9041461052942|[32.7391429383803...|
| 570.2004089636196|[33.9877728956856...|
| 427.1993848953282|[31.9365486184489...|
| 492.6060127179966|[33.9925727749537...|
| 522.3374046069357|[33.8793608248049...|
| 408.6403510726275|[29.5324289670579...|
| 573.4158673313865|[33.1903340437226...|
| 470.4527333009554|[32.3879758531538...|
| 461.7807421962299|[30.7377203726281...|
|457.84769594494855|[32.1253868972878...|
|407.70454754954415|[32.3388993230671...|
| 452.3156754800354|[32.1878120459321...|
|  605.061038804892|[32.6178560628234...|
+------------------+--------------

In [20]:
val lr = new LinearRegression()
val lrModel = lr.fit(output)

lr: org.apache.spark.ml.regression.LinearRegression = linReg_80a1d54ae1ef
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = LinearRegressionModel: uid=linReg_80a1d54ae1ef, numFeatures=4


In [21]:
val trainingSummary = lrModel.summary

trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@16201a72


In [27]:
trainingSummary.residuals.show()
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"MSE: ${trainingSummary.meanSquaredError}")
println(s"r2: ${trainingSummary.r2}")

+-------------------+
|          residuals|
+-------------------+
| -6.788234090018818|
| 11.841128565326073|
| -17.65262700858966|
| 11.454889631178617|
| 7.7833824373080915|
|-1.8347332184773677|
|  4.620232401352382|
| -8.526545950978175|
| 11.012210896516763|
|-13.828032682158891|
| -16.04456458615175|
|  8.786634365463442|
| 10.425717191807507|
| 12.161293785003522|
|  9.989313714461446|
| 10.626662732649379|
|  20.15641408428496|
|-3.7708446586326545|
| -4.129505481591934|
|  9.206694655890487|
+-------------------+
only showing top 20 rows

RMSE: 9.923256785022229
MSE: 98.47102522148971
r2: 0.9843155370226727


In [28]:
trainingSummary.predictions.show()

+------------------+--------------------+------------------+
|             label|            features|        prediction|
+------------------+--------------------+------------------+
| 587.9510539684005|[34.4972677251122...| 594.7392880584193|
| 392.2049334443264|[31.9262720263601...| 380.3638048790003|
|487.54750486747207|[33.0009147556426...|505.20013187606173|
| 581.8523440352177|[34.3055566297555...| 570.3974544040391|
| 599.4060920457634|[33.3306725236463...| 591.6227096084554|
|  637.102447915074|[33.8710378793419...| 638.9371811335513|
| 521.5721747578274|[32.0215955013870...|  516.951942356475|
| 549.9041461052942|[32.7391429383803...| 558.4306920562724|
| 570.2004089636196|[33.9877728956856...| 559.1881980671028|
| 427.1993848953282|[31.9365486184489...| 441.0274175774871|
| 492.6060127179966|[33.9925727749537...| 508.6505773041483|
| 522.3374046069357|[33.8793608248049...| 513.5507702414723|
| 408.6403510726275|[29.5324289670579...|   398.21463388082|
| 573.4158673313865|[33.